In [2]:
import os
import glob
import pandas as pd

# Working Directory

In [2]:
#from https://www.tutorialspoint.com/python/os_chdir.htm
#first retrieving working directory
#retval = os.getcwd()
#print ("Current working directory %s" % retval)
#returned: Current working directory /Users/jenniferbadger/Documents/GitHub/229project

#now changing working directory to where CSVs are located
#os.chdir("/Users/jenniferbadger/Documents/GitHub/229project/PAprimary")
#retval = os.getcwd()
#print ("Current working directory %s" % retval)
#returned: Current working directory /Users/jenniferbadger/Documents/GitHub/229project/PAprimary

Current working directory /Users/jenniferbadger/Documents/GitHub/229project
Current working directory /Users/jenniferbadger/Documents/GitHub/229project/PAprimary


# CSV Wrangling

These CSVs were exported from Purple Air's data download tool and contain PM 2.5 24 hour average data for 2019. 

The sensors were chosen using the Purple Air interactive map to subset first on a portion of greater LA that would include most of the HOLC boundaries and on sensors with names A-C that began delivering data during or before 2019. The reason for subsetting is that I ran into issues with hardware memory, limited interaction capability with regard to which CSVs I could choose (I had to download primary and secondary datasets for each sensor although the secorndary datasets were useless) and considering historical data was unavailable via the API. Names are defined by users and are not associated with location so are spatially random. Importantly, this subset yielded California Clean Air Coalition and AQMD/NASA sensors which are well distributed; eg. Echo Park, Pacoima, Brentwood, Inglewood, Montebello, Arcadia, Pacific Palisades to name a few. 

This subset yielded observations for some or all of the days in 2019 for 116 different point locations throughout greater Los Angeles.

https://api.purpleair.com
https://www.purpleair.com/sensorlist

In [3]:
#from https://sempioneer.com/python-for-seo/how-to-combine-multiple-csv-files-in-python/
#file_extension = '.csv'
#all_filenames = [i for i in glob.glob(f"*{file_extension}")]

#checking correct files are retieved
#print(f"These are all of the filenames ending in .csv {all_filenames}.")

These are all of the filenames ending in .csv ['Brentwood School West Campus (outside) (34.061087 -118.47584) Primary 1440_minute_average 01_01_2019 12_31_2019.csv'].


In [4]:
# prelim to concatenating two to see if we lose row 0 which contains location information
#all_filenames[0]

#cheking encoding of PA CSVs
#with open('Brentwood School West Campus (outside) (34.061087 -118.47584) Primary 1440_minute_average 01_01_2019 12_31_2019.csv') as q:
    #print(q)

#adapting code with encoding='UTF-8'    
#df = pd.read_csv('Brentwood School West Campus (outside) (34.061087 -118.47584) Primary 1440_minute_average 01_01_2019 12_31_2019.csv', 
                 #delimiter=',', encoding='UTF-8')

#print(df.shape) # shape is correct! but missing important location info

<_io.TextIOWrapper name='Brentwood School West Campus (outside) (34.061087 -118.47584) Primary 1440_minute_average 01_01_2019 12_31_2019.csv' mode='r' encoding='UTF-8'>
(248, 10)


In [5]:
#testing concatenating three csvs
#combined_csv_data = pd.concat([pd.read_csv(f, delimiter=',', encoding='UTF-8') for f in all_filenames])

#combined_csv_data.head(-50) #concatenates 

,created_at,PM1.0_CF1_ug/m3,PM2.5_CF1_ug/m3,PM10.0_CF1_ug/m3,UptimeMinutes,RSSI_dbm,Temperature_F,Humidity_%,PM2.5_ATM_ug/m3,Unnamed: 9
0,2019-08-01 00:00:00 UTC,10.86,34.12,53.79,50.66,-53.56,84.3,27.37,23.32,NaN
1,2019-08-01 00:00:00 UTC,10.86,34.12,53.79,50.66,-53.56,84.3,27.37,23.32,NaN
2,2019-08-02 00:00:00 UTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-08-02 00:00:00 UTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-08-03 00:00:00 UTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
193,2019-11-07 00:00:00 UTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,2019-11-08 00:00:00 UTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,2019-11-09 00:00:00 UTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,2019-11-10 00:00:00 UTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import re


path = r'/Users/jenniferbadger/Documents/GitHub/229project/PAprimary' # use your path
all_files = glob.glob(path + "/*.csv")


li = [] #only need this if appending to list instead of df
#https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

for filename in all_files:
    #reads csv into a dataframe
    df = pd.read_csv(filename, index_col=None,encoding='UTF-8', delimiter=',')
    #print(filename)
    filename = filename.split('/')
    filename = filename[-1]
    #print(filename)
    #adds csv extention- may be unnecessary
    #file_extension = '.csv'
    #converts file name to a i string list
    #filename = glob.glob(f"*{file_extension}")
    #adds file name to column
    df['filename'] = filename
    #print(df['filename'])
    #creates a list of parsed out strings inside parenthesis
    parenth = re.findall('\(([^)]+)', filename)
    #creates a 2 string list of lat, long
    lat_long = parenth[-1].split(" ")
    #extracts lat/long from list assignes them to column
    #and coverts them to float
    df['lat'] = lat_long[0] #astype(float)
    df['lat'] = df['lat'].astype(float)
    df['long'] = lat_long[-1] 
    df['long'] = df['long'].astype(float)
    #replaces NaN with zeroes
    df['PM2.5_ATM_ug/m3']= df['PM2.5_ATM_ug/m3'].fillna(0)
    df['PM2.5_CF1_ug/m3'] = df['PM2.5_CF1_ug/m3'].fillna(0)
    #filters for rows where Pm 2.5 values don't equal zero
    df = df[(df['PM2.5_ATM_ug/m3'] != 0)]
    #df = df[(df['PM2.5_CF1_ug/m3'] != 0)]
    #takes mean pm2.5
    df['PM2.5 ATM Mean'] = df['PM2.5_ATM_ug/m3'].mean()
    df['PM2.5 CF1 Mean'] = df['PM2.5_CF1_ug/m3'].mean()
    #df['PM2.5 ATM Mean'] = df[(df['PM2.5_ATM_ug/m3'] != 0)].mean()
    #df['PM2.5 CF1 Mean'] = df[(df['PM2.5_CF1_ug/m3']!= 0)].mean()
    
    #PM25_ATM_Mean = df[(df['PM2.5_ATM_ug/m3'] != 0)].mean()
    #PM25_CFI_Mean = df[(df['PM2.5_CF1_ug/m3']!= 0)].mean()
    #appends to df
    #df= df.append(df)
    #alt way to append to list
    
    #I only need the values from row 0, columns 11-14 to concat into one csv
    #drops unneeded columns
    try:
        df.drop(['PM1.0_CF1_ug/m3', 'PM2.5_CF1_ug/m3', 'PM10.0_CF1_ug/m3', 'UptimeMinutes', 
             'RSSI_dbm', 'Temperature_F', 'Humidity_%', 'PM2.5_ATM_ug/m3', 'Unnamed: 9'],
            inplace=True, axis = 1) 
    except:
        df.drop(['PM1.0_CF1_ug/m3', 'PM2.5_CF1_ug/m3', 'PM10.0_CF1_ug/m3', 'UptimeMinutes', 
             'RSSI_dbm', 'Temperature_F', 'Humidity_%', 'PM2.5_ATM_ug/m3'],
            inplace=True, axis = 1) 
        continue
    df = df.reset_index(drop=True)
    #print(df)
    try:
        newdf = df.iloc[0]
    except:
        newdf = df.iloc[-1]
        continue
    #converting series to df and flipping on its end
    newdf = pd.DataFrame(newdf).transpose()
    #newdf['PM2.5 ATM Mean'] = PM25_ATM_Mean
    #newdf['PM2.5 CF1 Mean'] = PM25_CFI_Mean
    li.append(newdf)
    #bigdf= bigdf.append(newdf)
    #alt way to convert series to df
    #newdf = newdf.to_frame()

final_frame = pd.concat(li, axis=0, ignore_index=True) 
final_frame.head(-10)

#newdf.head(70)
#final_frame = pd.concat([newdf], axis=0, ignore_index=True) 


#some code for spllitting on parenthesis
#s = 'Name(something)'
#re.search('\(([^)]+)', s).group(1)
#'something'

#https://stackoverflow.com/questions/45947887/python-looping-through-csv-files-and-their-columns
#for i in numFiles:
    #file = open(os.path.join(pathName, i), "rU")
    #reader = csv.reader(file, delimiter=',')
    #for row in reader:
        #for column in row:
            #print(column)
            #if column=="SPECIFIC VALUE":
                #do stuf                

,created_at,filename,lat,long,PM2.5 ATM Mean,PM2.5 CF1 Mean
0,2019-03-16 00:00:00 UTC,CCA 61st and King (outside) (33.98329 -118.183...,33.98329,-118.183926,16.995496,18.172958
1,2019-03-16 00:00:00 UTC,CCA 52nd Dr and Heliotrope (outside) (33.98995...,33.98995,-118.178093,16.328882,17.597437
2,2019-03-26 00:00:00 UTC,Adams Square (inside) (inside) (34.133594 -118...,34.133594,-118.241213,12.812237,14.105855
3,2019-03-27 00:00:00 UTC,CSUN Live Oak (outside) (34.238179 -118.528333...,34.238179,-118.528333,13.484826,13.641279
4,2019-01-09 00:00:00 UTC,6th St (inside) (34.020076 -118.49534) Primary...,34.020076,-118.49534,6.793395,7.069488
...,...,...,...,...,...,...
95,2019-04-10 00:00:00 UTC,CCA 56th and Atlantic (outside) (33.990066 -11...,33.990066,-118.185125,17.797489,19.23635
96,2019-05-06 00:00:00 UTC,COI5467 (outside) (34.085285 -117.960899) Prim...,34.085285,-117.960899,29.841667,37.285972
97,2019-01-15 00:00:00 UTC,10 (outside) (34.167005 -118.03768) Primary 14...,34.167005,-118.03768,54.275,79.555
98,2019-01-01 00:00:00 UTC,Bel Air (outside) (34.094154 -118.462079) Prim...,34.094154,-118.462079,11.429668,11.857695


In [74]:
#all_files[1]

'/Users/jenniferbadger/Documents/GitHub/229project/PAprimary/CSUN Live Oak (outside) (34.238179 -118.528333) Primary 1440_minute_average 01_01_2019 12_31_2019.csv'